# Предобработка данных

In [1]:
import pandas as pd
import numpy as np
import subfunc as sf
from impyute.imputation.cs import mice

In [2]:
RAW_DATA = 'data/evras.csv'
POST_PROCESSING_DATA = 'data/post_evras.csv'
SOURCE_DATA = 'source/'
pd.options.display.max_columns = 90
target_columns = ['химшлак последний Al2O3',
                  'химшлак последний CaO',
                  'химшлак последний R',
                  'химшлак последний SiO2']

Информация о данных

In [3]:
data = pd.read_csv(RAW_DATA, sep=';', index_col='index')
data.describe()

,nplv,DT,МАРКА,ПРОФИЛЬ,t вып-обр,t обработка,t под током,t продувка,ПСН гр.,чист расход C,чист расход Cr,чист расход Mn,чист расход Si,чист расход V,температура первая,температура последняя,Ar (интенс.),N2 (интенс.),эл. энергия (интенс.),произв жидкая сталь,произв количество обработок,произв количество плавок,произв количество плавок (цел),расход газ Ar,расход газ N2,расход C пров.,сыпуч известь РП,сыпуч кварцит,сыпуч кокс пыль УСТК,сыпуч кокс. мелочь (сух.),сыпуч кокс. мелочь КМ1,сыпуч шпат плав.,ферспл CaC2,ферспл FeMo,ферспл FeSi-75,ферспл FeV азот.,ферспл FeV-80,ферспл Mn5Si65Al0.5,ферспл Ni H1 пласт.,ферспл SiMn18,ферспл ферванит,ферспл фх850А,эл. энергия,химсталь первый Al_1,химсталь первый C_1,химсталь первый Cr_1,химсталь первый Cu_1,химсталь первый Mn_1,химсталь первый Mo_1,химсталь первый N_1,химсталь первый Ni_1,химсталь первый P_1,химсталь первый S_1,химсталь первый Si_1,химсталь первый Ti_1,химсталь первый V_1,химсталь последний Al,химсталь последний C,химсталь последний Ca,химсталь последний Cr,химсталь последний Cu,химсталь последний Mn,химсталь последний Mo,химсталь последний N,химсталь последний Ni,химсталь последний P,химсталь последний S,химсталь последний Si,химсталь последний Ti,химсталь последний V,химшлак первый Al2O3_1,химшлак первый CaO_1,химшлак первый FeO_1,химшлак первый MgO_1,химшлак первый MnO_1,химшлак первый R_1,химшлак первый SiO2_1,химшлак последний Al2O3,химшлак последний CaO,химшлак последний FeO,химшлак последний MgO,химшлак последний MnO,химшлак последний R,химшлак последний SiO2
count,7041,7041,7041,7041,6444,6440,6440,6444,5249,6435,5883,6431,6431,6178,6436,6436,6426,1389,6440,6444,6444,6444,6444,6426,1389,724,4770,13,6422,7,6307,5910,6244,33,5,383,6179,6400,82,6398,1017,5883,6440,6444,6444,6444,6444,6444,6443,6443,6444,6444,6444,6444,6444,6444,6444,6444,6218,6444,6444,6444,6444,6442,6444,6444,6444,6444,6444,6444,4742,6444,6444,6443,6444,6444,4742,4742,6444,6444,6443,6444,6444,4742
unique,7041,7041,18,18,82,816,1155,831,4296,6243,3115,6320,6265,4645,154,109,6423,1389,6362,226,3,1,1,5556,1341,593,469,1,307,7,691,197,7,28,2,83,692,399,80,528,199,398,3132,6,417,371,101,404,24,18,115,18,34,384,7,28,20,226,32,206,100,261,24,17,112,19,27,256,5,54,73,298,54,236,152,32,149,70,229,16,192,61,16,116
top,г683863??;?,2071-01-21 12:44:00,Э76ХФ,Р65,"6,0","43,0333333","19,6833333","43,0333333","2,0","0,406765","0,104904","0,132318","0,184723","0,0391018","1571,0","1579,0","14,9067961","12,6160428","12600,0","115,4","1,0","1,0","1,0","13,192","9,016","0,019448","0,063","0,02","0,05","0,417","0,384","0,041","0,02","0,021014","0,05","0,024306","0,050094","0,128","0,04919","0,219","0,025897","0,214","4199,0","0,002","0,435","0,308","0,1","0,672","0,006","0,01","0,071","0,008","0,019","0,449","0,002","0,002","0,003","0,762","0,0002","0,418","0,103","0,789","0,006","0,012","0,071","0,009","0,015","0,566","0,003","0,037","3,7","53,4","0,4","11,3","0,11","2,2","21,7","4,4","57,6","0,5","8,7","0,08","2,2","26,0"
freq,1,1,4953,6427,227,53,32,56,526,3,17,3,2,9,134,282,2,1,4,124,6391,6444,6444,5,3,4,87,13,5873,1,34,250,6137,3,4,14,81,61,2,45,23,60,11,4656,62,91,236,73,1783,1751,308,1410,904,105,3727,2929,4526,166,1816,660,240,353,1590,1305,324,1363,1076,229,4764,1793,263,58,1056,96,470,915,100,243,102,1235,88,723,1483,142


Нахождение самой популярной марки

In [4]:
top_mark = data.describe().loc["top", "МАРКА"]
print("Самая популярная марка - ", top_mark)

Самая популярная марка -  Э76ХФ


Процент пропущенных значений в датасете

In [5]:
data.isnull().sum().sort_values(ascending= False).apply(lambda x : x / data.shape[0])[:15]

ферспл FeSi-75                0.999290
сыпуч  кокс. мелочь (сух.)    0.999006
сыпуч кварцит                 0.998154
ферспл FeMo                   0.995313
ферспл  Ni H1 пласт.          0.988354
ферспл FeV азот.              0.945604
расход C пров.                0.897174
ферспл  ферванит              0.855560
расход газ  N2                0.802727
N2 (интенс.)                  0.802727
химшлак первый SiO2_1         0.326516
химшлак последний Al2O3       0.326516
химшлак первый Al2O3_1        0.326516
химшлак последний SiO2        0.326516
сыпуч известь РП              0.322539
dtype: float64

Перевод даты во время изготовления сплава

In [6]:
data['DT'] = sf.date_to_interval(data['DT'])

Оставляем колонки с > 65% известных данных

In [7]:
dropped_data = sf.drop_columns(data,0.65)

Удаленные колонки: N2 (интенс.), расход газ  N2, расход C пров., сыпуч кварцит, сыпуч  кокс. мелочь (сух.), ферспл FeMo, ферспл FeSi-75, ферспл FeV азот., ферспл  Ni H1 пласт., ферспл  ферванит


In [8]:
#part_nan = dropped_data.isnull().sum().sort_values(ascending= False).apply(lambda x : x / data.shape[0])

Зануляем показатели трети хим анализа

In [9]:
listd = []
for column in data.columns:
    if 'хим' in column:
        listd.append(column)

In [10]:
dropped_data[listd] = dropped_data[listd].fillna(0)

C:\Users\vasya\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
str_columns = ['ПРОФИЛЬ','МАРКА','nplv','DT']
all_columns = dropped_data.columns.to_list()
result_columns = list(set(all_columns) - set(str_columns))

In [13]:
trunc_data = dropped_data[result_columns].apply(lambda x:
                                                 x.str.replace(',', '.').astype(float))

In [14]:
from impyute.imputation.cs import mice

imputed_training = mice(trunc_data.values)

KeyboardInterrupt: 

In [ ]:
dropped_data[result_columns] = imputed_training

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


plt.figure(figsize=(7, 7))
sns.distplot(dropped_data['химшлак последний Al2O3'], color='g', bins=100, hist_kws={'alpha': 0.4}, vertical=True)
#plt.show()
plt.savefig(f'{SOURCE_DATA}target_range_1.png')

In [ ]:
plt.figure(figsize=(7, 7))
sns.distplot(dropped_data['химшлак последний CaO'], color='r', bins=100, hist_kws={'alpha': 0.4}, vertical=True)
#plt.show()
plt.savefig(f'{SOURCE_DATA}target_range_2.png')

In [ ]:
plt.figure(figsize=(7, 7))
sns.distplot(dropped_data['химшлак последний R'], color='g', bins=100, hist_kws={'alpha': 0.4}, vertical=True)
plt.show()
plt.savefig(f'{SOURCE_DATA}target_range_3.png')

In [ ]:
plt.figure(figsize=(7, 7))
sns.distplot(dropped_data['химшлак последний SiO2'], color='r', bins=100, hist_kws={'alpha': 0.4}, vertical=True)
plt.show()
plt.savefig(f'{SOURCE_DATA}target_range_4.png')

In [ ]:
corr_data = dropped_data.drop(columns=target_columns).corr()

plt.figure(figsize=(52, 50))

sns.heatmap(corr_data[(corr_data >= 0.5) | (corr_data <= -0.4)],
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True)
plt.savefig(f'{SOURCE_DATA}correletion_map.png')

In [ ]:
dropped_data.to_csv(POST_PROCESSING_DATA, sep='\t', index=False)